In [1]:
#Import modules
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import optimizers
from keras import models
from keras import layers
import tensorflow as tf
from keras import regularizers

### Download dataset and re-scale values

In [2]:
#Load dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
#Scale values so that they are in [0-1] interval
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
#Split training dataset to generate validation dataset
x_val = x_train[40000:]
x_train = x_train[:40000]
y_val = y_train[40000:]
y_train = y_train[:40000]

### Instantiate pre-trained convolutional base

In [4]:
#Load weights learned from training on image net dataset
#Don't include top (will add our own dense classifier)
convolutional_base = VGG16(weights = 'imagenet', include_top = False)

### Define parameters for Grid Search of optimal regularization parameters for model

In [5]:
#Grid search parameters
dropouts = [0, 0.1, 0.3, 0.5]
regularization = [regularizers.l2(0.001), None]

### Define function to generate models for Grid Search

In [6]:
#Define function to generate model
def grid_search_model(drop, reg):
    if reg != None:
        model = models.Sequential()
        model.add(convolutional_base)
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dense(256, kernel_regularizer= reg, activation='relu'))
        model.add(layers.Dropout(drop))
        model.add(layers.Dense(256, kernel_regularizer= reg, activation='relu'))
        model.add(layers.Dropout(drop))
        model.add(layers.Dense(64, kernel_regularizer= reg, activation='relu'))
        model.add(layers.Dense(64, kernel_regularizer= reg, activation='relu'))
        model.add(layers.Dense(64, kernel_regularizer= reg, activation='relu'))
        model.add(layers.Dense(10, kernel_regularizer= reg, activation='softmax'))
        
    else:
        model = models.Sequential()
        model.add(convolutional_base)
        model.add(layers.GlobalAveragePooling2D())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(drop))
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(drop))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(10, activation='softmax'))
        
    return model

### Perform Grid Search

In [7]:
results = []

for i in dropouts:
    for j in regularization:
        
        convolutional_base = VGG16(weights = 'imagenet', include_top = False)
        
        gs_model = grid_search_model(i,j)
        
        convolutional_base.trainable = False
        
        gs_model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = optimizers.RMSprop(learning_rate = 2e-5),
                 metrics = ['acc'])
        
        history_dense_top = gs_model.fit(
                                          x = x_train,
                                          y = y_train,
                                          steps_per_epoch=100,
                                          epochs=30,
                                          validation_steps=50)
        
        val_loss_top, val_acc_top = gs_model.evaluate(x = x_val, y = y_val, steps=100)

        convolutional_base.trainable = True

        for layer in convolutional_base.layers:
            if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
                layer.trainable = True
            else:
                layer.trainable = False
        
        
        gs_model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = optimizers.RMSprop(learning_rate = 1e-5),
                 metrics = ['acc'])
        
        history_final = gs_model.fit(
              x = x_train,
              y = y_train,
              steps_per_epoch=100,
              epochs=20)
        
        val_loss_final, val_acc_final = gs_model.evaluate(x = x_val, y = y_val, steps=100)
        
        results.append([i, j, val_acc_top, val_acc_final])

Epoch 1/30
100/100 [==============================] - 6s 21ms/step - loss: 3.1201 - acc: 0.1408
Epoch 2/30
100/100 [==============================] - 2s 21ms/step - loss: 3.0358 - acc: 0.1975
Epoch 3/30
100/100 [==============================] - 2s 21ms/step - loss: 2.9531 - acc: 0.2572
Epoch 4/30
100/100 [==============================] - 2s 21ms/step - loss: 2.8599 - acc: 0.3074
Epoch 5/30
100/100 [==============================] - 2s 21ms/step - loss: 2.7596 - acc: 0.3387
Epoch 6/30
100/100 [==============================] - 2s 21ms/step - loss: 2.6577 - acc: 0.3641
Epoch 7/30
100/100 [==============================] - 2s 21ms/step - loss: 2.5591 - acc: 0.3873
Epoch 8/30
100/100 [==============================] - 2s 21ms/step - loss: 2.4682 - acc: 0.4109
Epoch 9/30
100/100 [==============================] - 2s 21ms/step - loss: 2.3891 - acc: 0.4291
Epoch 10/30
100/100 [==============================] - 2s 21ms/step - loss: 2.3205 - acc: 0.4424
Epoch 11/30
100/100 [==================

In [8]:
#Print best regularization parameter combination
current_best_params = None
current_best_accuracy = 0

for result in results:
    if result[3] > current_best_accuracy:
        current_best_accuracy = result[3]
        current_best_params = (result[0], result[1])
                               
print(current_best_params)
print(current_best_accuracy)

(0, None)
0.7229999899864197


### Train final model

In [9]:
#Can now train on all data
import numpy as np
x_train = np.concatenate((x_train, x_val))
y_train = np.append(y_train, y_val)

In [12]:
convolutional_base = VGG16(weights = 'imagenet', include_top = False)
        
final_model = models.Sequential()
final_model.add(convolutional_base)
final_model.add(layers.GlobalAveragePooling2D())
final_model.add(layers.Dense(256, activation='relu'))
final_model.add(layers.Dense(256, activation='relu'))
final_model.add(layers.Dense(64, activation='relu'))
final_model.add(layers.Dense(64, activation='relu'))
final_model.add(layers.Dense(64, activation='relu'))
final_model.add(layers.Dense(10, activation='softmax'))
        
convolutional_base.trainable = False

final_model.compile(loss = 'sparse_categorical_crossentropy',
         optimizer = optimizers.RMSprop(learning_rate = 2e-5),
         metrics = ['acc'])

history_dense_top = final_model.fit(
                                  x = x_train,
                                  y = y_train,
                                  steps_per_epoch=100,
                                  epochs=30,
                                  validation_steps=50)

convolutional_base.trainable = True

for layer in convolutional_base.layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False


final_model.compile(loss = 'sparse_categorical_crossentropy',
         optimizer = optimizers.RMSprop(learning_rate = 1e-5),
         metrics = ['acc'])

history_final = final_model.fit(
      x = x_train,
      y = y_train,
      steps_per_epoch=100,
      epochs=100)

test_loss_final, test_acc_final = final_model.evaluate(x = x_test, y = y_test, steps=100)

Epoch 1/30
100/100 [==============================] - 3s 25ms/step - loss: 2.2796 - acc: 0.1443
Epoch 2/30
100/100 [==============================] - 2s 25ms/step - loss: 2.2123 - acc: 0.2212
Epoch 3/30
100/100 [==============================] - 3s 25ms/step - loss: 2.1353 - acc: 0.2771
Epoch 4/30
100/100 [==============================] - 3s 25ms/step - loss: 2.0496 - acc: 0.3236
Epoch 5/30
100/100 [==============================] - 2s 25ms/step - loss: 1.9595 - acc: 0.3510
Epoch 6/30
100/100 [==============================] - 3s 25ms/step - loss: 1.8707 - acc: 0.3828
Epoch 7/30
100/100 [==============================] - 3s 25ms/step - loss: 1.7887 - acc: 0.4067
Epoch 8/30
100/100 [==============================] - 3s 26ms/step - loss: 1.7186 - acc: 0.4236
Epoch 9/30
100/100 [==============================] - 3s 26ms/step - loss: 1.6594 - acc: 0.4377
Epoch 10/30
100/100 [==============================] - 3s 25ms/step - loss: 1.6105 - acc: 0.4510
Epoch 11/30
100/100 [==================

In [13]:
#Print final model accuracy on testing dataset
print(test_acc_final)

0.7402999997138977


Final model achieved an accuracy score of 74% on the testing dataset.